In [1]:
import numpy as np

# 1


In [2]:
def _simplex_1p(A, c, bs_i, eps=1e-6):
    """
    max. c^T x
    s.t. A[:][:-1] x == A[:][-1], x >= 0, A[i][bs_i[i]] != 0, x[!bs_i] == 0
    """
    m, n = A.shape
    assert (m < n) and ((n,) == (c.shape)) and ((m,) == (bs_i.shape))
    x = np.zeros(n - 1)
    for i in range(m):
        assert (0 <= bs_i[i] < (n - 1)) and abs(A[i][bs_i[i]]) > eps
        A[i] /= A[i][bs_i[i]]
        c -= c[bs_i[i]] * A[i]
        for j in range(m):
            if i != j:
                A[j] -= A[j][bs_i[i]] * A[i]
    for i in range(m):
        assert A[i][-1] >= 0
        x[bs_i[i]] = A[i][-1]
    while c[idx1 := np.argmin(c[:-1])] < 0:
        x[idx1] = np.inf
        idx2 = None
        for i in range(m):
            if abs(A[i][idx1]) > eps:
                if idx2 is None:
                    idx2 = i
                tmp = A[i][-1] / A[i][idx1]
                if tmp >= 0 and tmp < x[idx1]:
                    idx2 = i
                    x[idx1] = tmp
        if idx2 is not None:
            if x[idx1] == np.inf:
                x[bs_i[idx2]] = np.inf
            else:
                x[bs_i[idx2]] = 0
                bs_i[idx2] = idx1
        if x[idx1] == np.inf:
            break
        A[idx2] /= A[idx2][idx1]
        c -= c[idx1] * A[idx2]
        for i in range(m):
            if i != idx2:
                A[i] -= A[i][idx1] * A[idx2]
                x[bs_i[i]] = A[i][-1]
    return c[-1], x

In [3]:
A = np.array([[2, 1], [1, 1], [1, 0]])
b = np.array([20, 18, 8])
c = np.array([7, 4])
m, n = A.shape
display(
    _simplex_1p(
        np.concatenate((A, np.eye(m), b.reshape(-1, 1)), axis=1),
        np.concatenate((-c, np.zeros(m + 1))),
        np.arange(n, n + m),
    )
)

(np.float64(78.0), array([ 2., 16.,  0.,  0.,  6.]))

# 2
